In [ ]:

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import spearmanr

import os


# Paths for data and adjacency matrices
hmm_dir = '/Volumes/ARCHIVES/thesis_pipeline/data/group_level_HMM'
adjacency_dir = '/Volumes/ARCHIVES/thesis_pipeline/data/group_level_adjacency_matrices'
figures_dir = '/Volumes/ARCHIVES/thesis_pipeline/figures/adjacency_matrices'


os.makedirs(adjacency_dir, exist_ok=True)
os.makedirs(figures_dir , exist_ok=True)

roi_names = ['PTL', 'ATL', 'AG', 'IFG', 'MFG', 'IFGorb']

In [ ]:
def create_adjacency_matrix(n_TRs, event_bounds):
    print(f"Building adjacency matrix for {n_TRs} TRs...")
    adjacency_matrix = np.zeros((n_TRs, n_TRs))

    # Augment boundaries with start and end of the sequence
    bounds_augmented = np.concatenate(([0], event_bounds, [n_TRs]))
    print(f"  Event boundaries augmented with start and end: {bounds_augmented[:10]}... (showing first 10)")

    # Fill the adjacency matrix for each segment
    for i in range(len(bounds_augmented) - 1):
        start, end = bounds_augmented[i], bounds_augmented[i + 1]
        adjacency_matrix[start:end, start:end] = 1
        print(f"  Filled segment from {start} to {end}")

    print("Adjacency matrix construction complete.")
    return adjacency_matrix


In [ ]:
# Construct and save adjacency matrices for each ROI
for roi in roi_names:
    event_bounds_path = os.path.join(hmm_dir, f'{roi}_event_bounds.npy')
    group_data_path = os.path.join('/Volumes/ARCHIVES/thesis_pipeline/data/group_SRM_data', f'{roi}_group_SRM_data.npy')
    
    # Check if event boundaries and group data exist
    if os.path.exists(event_bounds_path) and os.path.exists(group_data_path):
        event_bounds = np.load(event_bounds_path)
        n_TRs = np.load(group_data_path).shape[1]  # TRs are the second dimension of group data
        
        # Construct the adjacency matrix
        adjacency_matrix = create_adjacency_matrix(n_TRs, event_bounds)
        
        # Save the adjacency matrix
        adjacency_save_path = os.path.join(adjacency_dir, f'{roi}_adjacency_matrix.npy')
        np.save(adjacency_save_path, adjacency_matrix)
        
        print(f"Saved adjacency matrix for {roi} with shape {adjacency_matrix.shape}")
    else:
        print(f"Event boundaries or group data for {roi} not found.")

In [ ]:
for roi in roi_names:
    adjacency_path = os.path.join(adjacency_dir, f'{roi}_adjacency_matrix.npy')
    
    if os.path.exists(adjacency_path):
        adjacency_matrix = np.load(adjacency_path)
        
        # Plot the adjacency matrix without tick labels
        plt.figure(figsize=(10, 8))
        sns.heatmap(adjacency_matrix, cmap='Blues', cbar=True)
        plt.title(f'Adjacency Matrix - {roi}', fontsize=16, weight='bold')
        plt.xlabel('TRs', fontsize=14, weight='bold')
        plt.ylabel('TRs', fontsize=14, weight='bold')
        
       
        plt.xticks([])
        plt.yticks([])
        
       
        plot_save_path = os.path.join(figures_dir, f'{roi}_adjacency_matrix.png')
        plt.tight_layout()
        plt.savefig(plot_save_path, dpi=300)
        plt.close()
        
        print(f"Saved adjacency matrix plot for {roi} at {plot_save_path}")
        
    else:
        print(f"Adjacency matrix file for {roi} not found.")

In [ ]:
isc_dir = '/Volumes/ARCHIVES/thesis_pipeline/data/ISC_vectors'

def compute_hmm_adjacency_vector(adjacency_matrix):
    return np.mean(adjacency_matrix, axis=1)

In [ ]:
# Main analysis loop for each ROI
for roi in roi_names:
    isc_path = os.path.join(isc_dir, f'{roi}_ISC_vector.npy')
    adjacency_path = os.path.join(adjacency_dir, f'{roi}_adjacency_matrix.npy')
    
    if os.path.exists(isc_path) and os.path.exists(adjacency_path):
        # Load ISC vector from IS-RSA and HMM adjacency matrix
        isc_vector = np.load(isc_path)
        hmm_adjacency_matrix = np.load(adjacency_path)
        
        # Convert HMM adjacency matrix to a vector
        hmm_adjacency_vector = compute_hmm_adjacency_vector(hmm_adjacency_matrix)
        
        # Convert both vectors to Adjacency objects for nltools
        isc_adjacency = Adjacency(isc_vector)
        hmm_adjacency = Adjacency(hmm_adjacency_vector)
        
        # Compute observed correlation
        observed_correlation = isc_adjacency.similarity(hmm_adjacency, metric='spearman')['correlation']
        print(f"Observed correlation for {roi}: {observed_correlation}")
        
        # Permutation testing
        permuted_correlations = []
        for _ in range(n_permutations):
            np.random.shuffle(isc_vector)  # Shuffle ISC vector for null distribution
            permuted_correlation = spearmanr(isc_vector, hmm_adjacency_vector)[0]
            permuted_correlations.append(permuted_correlation)
        
        # Calculate p-value
        permuted_correlations = np.array(permuted_correlations)
        p_value = (np.sum(permuted_correlations >= observed_correlation) + 1) / (n_permutations + 1)
        
        print(f"Permutation test p-value for {roi}: {p_value}")
    else:
        print(f"ISC vector or HMM adjacency matrix for {roi} not found.")